In [1]:
from connect_github_log import ConnDB
from mylib.mysql_tool import execute_sql
import pandas as pd

date_list = ['2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01',
             '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
             '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01',
             '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
             '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01',
             '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
             '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01',
             '2020-07-01', '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
             '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
             '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
             ]
weight_map = {
    "IssueCommentEvent": 1,
    "IssuesEvent": 2,
    "PullRequestReviewCommentEvent": 3,
    "PullRequestEvent": 4,
    "PushEvent": 5
}
year_list = ['2017', '2018', '2019', '2020', '2021']
repo_list = pd.DataFrame()
user_list = pd.DataFrame()
edges_list = pd.DataFrame(columns=['repo_i', 'repo_j', 'weight', 'date'])

# 获取 2017-2021内 仓库与用户

In [2]:
for year in year_list:
    conndb = ConnDB()
    conndb.sql = f"SELECT DISTINCT(repo_id), repo_name FROM year{year} WHERE org_login = 'Azure'"
    conndb.execute()
    repo_list = repo_list.append(conndb.df_rs)

    conndb.sql = f"SELECT DISTINCT(actor_id) FROM year{year} WHERE org_login = 'Azure' AND type in ('IssueCommentEvent', 'IssuesEvent', 'PullRequestReviewCommentEvent', 'PullRequestEvent','PushEvent')"
    conndb.execute()
    user_list = user_list.append(conndb.df_rs)
repo_list.drop_duplicates()
user_list.drop_duplicates()
print(repo_list, user_list)


              0                                                  1
0       4127088                         Azure/azure-sdk-for-python
1      35129048                       Azure/azure-mobile-apps-node
2      55558713                              Azure/Azure-Functions
3      70936151                         Azure/azure-iot-sdk-csharp
4      88594567                  Azure/azure-iot-connected-factory
...         ...                                                ...
1857  110286680                            Azure/autorest.megarepo
1858  141767103                       Azure/terraform-provider-jwt
1859  138939502               Azure/fluent-plugin-process-snmptrap
1860  155638073                            Azure/fluent-plugin-mdm
1861  310746722  Azure/Azure-AppServices-Diagnostics-KustoIngestor

[6574 rows x 2 columns]               0
0       4258913
1       4260261
2       4268349
3       7637700
4       7642256
...         ...
24190  48744965
24191  47611942
24192  48365571
24193  4643

# 建立边，并存储

In [4]:
for i in range(0, len(date_list)-1):
    edges_list = pd.DataFrame(columns=['repo_i', 'repo_j', 'weight', 'date'])
    # 偶有网络不稳定，抛出err
    # try:
    conndb = ConnDB()
    conndb.sql = f"SELECT DISTINCT(actor_id) FROM year{date_list[i][:4]} WHERE org_login = 'Azure' AND type in ('IssueCommentEvent', 'IssuesEvent', 'PullRequestReviewCommentEvent', 'PullRequestEvent','PushEvent') AND created_date >= '{date_list[i]}' AND created_date < '{date_list[i+1]}' GROUP BY actor_id HAVING COUNT( DISTINCT( repo_id)) >= 10"
    conndb.execute()
    user_list = conndb.df_rs
    print(user_list)
    for index, row in user_list.iterrows():
        # 获取该期间内所有有关user
        user_id = user_list.iat[index, 0]
        conndb = ConnDB()
        conndb.sql = f"SELECT repo_id, repo_name, type, actor_id, created_date FROM year{date_list[i][:4]} WHERE org_login = 'Azure' AND actor_id = {user_id} AND type in ('IssueCommentEvent', 'IssuesEvent', 'PullRequestReviewCommentEvent', 'PullRequestEvent','PushEvent') AND created_date >= '{date_list[i]}' AND created_date < '{date_list[i+1]}' "
        conndb.execute()
        res = conndb.df_rs
        # 同类操作聚合
        res = pd.DataFrame(res.groupby([0,2], as_index=False).count())
        print(res)
        for index_1, row_1 in res.iterrows():
            for index_2, row_2 in res.iterrows():
                if res.iat[index_1,0] != res.iat[index_2,0] and res.iat[index_1,1] == res.iat[index_2,1]:
                    edges_list = edges_list.append({'repo_i': res.iat[index_1,0], 'repo_j': res.iat[index_2,0], 'weight': weight_map[res.iat[index_1,1]] * res.iat[index_1, 2] , 'date': {date_list[i]}}, ignore_index=True)
    # except:
    #     print("db err!")
    print(edges_list)
    # 聚合运算
    weight_sum = pd.DataFrame(edges_list.groupby(['repo_i','repo_j'], as_index=False)['weight'].sum())
    print(weight_sum)
    # 存入mysql
    for index_3, row_3 in weight_sum.iterrows():
        repo_i = weight_sum.iat[index_3, 0]
        repo_j = weight_sum.iat[index_3, 1]
        weight = weight_sum.iat[index_3, 2]
        execute_sql(f'INSERT INTO edge VALUES ({repo_i}, {repo_j}, {weight}, "{date_list[i]}")')

          0
0   8782776
1   9868777
2   8030856
3   9287708
4  10118926
8782776             0                           1                              2  \
0     2950981    Azure/azure-sdk-for-node  PullRequestReviewCommentEvent   
1     2950981    Azure/azure-sdk-for-node  PullRequestReviewCommentEvent   
2     4127088  Azure/azure-sdk-for-python                    IssuesEvent   
3     4127088  Azure/azure-sdk-for-python                    IssuesEvent   
4     4127088  Azure/azure-sdk-for-python                    IssuesEvent   
..        ...                         ...                            ...   
176  51040886             Azure/azure-cli  PullRequestReviewCommentEvent   
177  51040886             Azure/azure-cli  PullRequestReviewCommentEvent   
178  51040886             Azure/azure-cli  PullRequestReviewCommentEvent   
179  51040886             Azure/azure-cli  PullRequestReviewCommentEvent   
180  56031054   Azure/azure-ruby-asm-core              IssueCommentEvent   

       